In [52]:
import shapefile
import sys
from json import dumps
from eviltransform import wgs2gcj

def clean_encoding(obj):
    if type(obj) is bytes:
        #return obj.decode(encoding='gb2312')
        return obj.decode(encoding='gb18030')
    return obj
def encoding(obj):
    if type(obj) is unicode:
        return obj.encode(encoding='utf8')
    return obj

def wgs2gcj_ext(geometry_point):
    '''
    geometry_point: lon, lat
    return geometry point: lon, lat
    '''
    precision = 1000000
    gcj_point = wgs2gcj(geometry_point[1], geometry_point[0])
    lon = int(gcj_point[1] * precision) * 1.0 / precision
    lat = int(gcj_point[0] * precision) * 1.0 / precision
    return [lon, lat]

def trans_coordinate(geometry):
    '''
    转坐标系
    '''
    coordinates = []
    geo_type = geometry['type']
    if geo_type == 'Point':
        point = geometry['coordinates']
        coordinates = wgs2gcj_ext(point)
    elif geo_type == 'LineString':
        for point in geometry['coordinates']:
            coordinates.append(wgs2gcj_ext(point))
    elif geo_type == 'Polygon':
        for points in geometry['coordinates']:
            new_points = []
            for point in points:
                new_points.append(wgs2gcj_ext(point))
            coordinates.append(new_points)
    elif geo_type == 'MultiPolygon':
        for polygons in geometry['coordinates']:
            new_polygons = []
            for points in polygons:
                new_points = []
                for point in points:
                    new_points.append(wgs2gcj_ext(point))
                new_polygons.append(new_points)
            coordinates.append(new_polygons)
    else:
        raise Exception("not supported shapeType: " + geo_type)
    geometry['coordinates'] = coordinates
    return geometry



In [76]:
reader = shapefile.Reader('/Users/yongyao.zhuang/Downloads/2018/link.beijing.2018/beijing_polygon.shp')
fields = reader.fields[:]

# write shapefile
w = shapefile.Writer()

# write fields
for field in fields:
    w.field(field[0], field[1], field[2], field[3])

# write shape
for sr in reader.shapeRecords():
    # clean encoding
    record = map(encoding, map(clean_encoding, sr.record))
    
    # mk shape
    shape = shapefile._Shape(sr.shape.shapeType)
    shape.parts = sr.shape.parts
    shape.points = map(wgs2gcj_ext, sr.shape.points)
    
    w.shapes().append(shape)
    w.record(*record)
    
w.save('/Users/yongyao.zhuang/tmp/1/1')
print 'done'

done
